In [22]:
%pip install cvzone
%pip install --upgrade mediapipe
%pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



## Importing libraries

In [23]:
import cv2
import mediapipe
import os
import matplotlib.pyplot as plt
import numpy as np
from cvzone.HandTrackingModule import HandDetector

help(cv2)

Help on package cv2:

NAME
    cv2 - OpenCV Python binary extension loader

PACKAGE CONTENTS
    config
    config-3
    cv2
    data (package)
    gapi (package)
    load_config_py2
    load_config_py3
    mat_wrapper (package)
    misc (package)
    typing (package)
    utils (package)
    version

SUBMODULES
    Error
    aruco
    barcode
    bgsegm
    bioinspired
    ccm
    colored_kinfu
    cuda
    datasets
    detail
    dnn
    dnn_superres
    dpm
    dynafu
    face
    fisheye
    flann
    ft
    hfs
    img_hash
    intensity_transform
    ipp
    kinfu
    large_kinfu
    legacy
    line_descriptor
    linemod
    mcc
    ml
    motempl
    multicalib
    ocl
    ogl
    omnidir
    optflow
    parallel
    phase_unwrapping
    plot
    ppf_match_3d
    quality
    rapid
    reg
    rgbd
    saliency
    samples
    segmentation
    stereo
    structured_light
    text
    videoio_registry
    videostab
    wechat_qrcode
    xfeatures2d
    ximgproc
    xphoto

DATA
  

## Defining variables

In [24]:
w, h = 980, 768
SlidesFolder = '../hand gesture presentation/images'
SlideNum = 0
#adjusting the size of the webcam on the presentation slides
cam_top = 2
cam_left = w - 2
cam_height = 150
cam_width = 250
gesture_activation = 340
keyEntered = False
keyCount = 0
#the key must be held down for 1 second which is equal to 20fps in this case
keyDelay = 20 
drawing = []
#assigning the value as -1 so that it turns to 0 when it's incremented
drawingCount = -1
drawingPoint = False


## Webcam setup

In [25]:
#adjusting the webcam to capture the video
cam = cv2.VideoCapture(0)
cam.set(3, w)
cam.set(4, h) 

True

## Retrieving the presentation images

In [26]:
#getting the list of images from the folder above
presSlides = os.listdir(SlidesFolder)

## Addition of Hand Detector

In [27]:
#adding a hand-detector to track the hands
tracker = HandDetector(detectionCon = 0.9, maxHands = 2)

In [28]:
while True:
    #read a frame from the variable cam
    success, img = cam.read()

    #flipping the webcam horizontally
    img = cv2.flip(img, 1)

    #importing the presentation images
    PptSlides = os.path.join(SlidesFolder, presSlides[SlideNum])
    currentSlide = cv2.imread(PptSlides)

    #detecting the hands in the webcam image
    palms, img = tracker.findHands(img)

    #creating the gesture activation line
    cv2.line(img, (0, gesture_activation),(w, gesture_activation), (238, 197, 160), 6)

    #checking to see if fingers are detected 
    if palms and keyEntered is False:

        #processing data about the initially detected palm including the number of fingers held up, the center 
        #coordinates of the palm and its index fingers' tip

        OnePalm = palms[0]
        Palmfingers = tracker.fingersUp(OnePalm)
        centerHorizontal, centerVertical = OnePalm['center']
        landmarkList = OnePalm['lmList']
        scaling_fact = 2.0

    #restraining and scaling the coordinate values in order to easily and smoothly move the cursor all over the slides 
        x_coord = int(np.interp(landmarkList[8][0], [w // 4, w], [0, w]) * scaling_fact) 
        y_coord = int(np.interp(landmarkList[8][1], [140, h - 140], [0, h]) * scaling_fact) 
        IndxFinger = x_coord, y_coord

       #making sure that the model only accepts gestures when they are above the gesture activation line
        if centerVertical <= gesture_activation:

           # First gesture to move to the previous slide  
           if Palmfingers == [1, 0, 0, 0, 0]:
              #if the slide number is greater than 0, then it will move to the previous slide
              if SlideNum > 0:
                 keyEntered = True 
                 SlideNum -= 1
                 drawing = []
                 drawingCount = -1
                 drawingPoint = False

           # Second gesture to move to the next slide
           if Palmfingers == [0, 1, 0, 0, 0]:
              #print("right") #COMMENT OR DELETE THIS LATER
              #checking to see if there is another slide after this one, if yes, proceed to the next one, or else, stay in the same slide
              if SlideNum < len(presSlides) - 1:
                 keyEntered = True
                 SlideNum += 1
                 drawing = []
                 drawingCount = -1
                 drawingPoint = False
            
        #Third gesture for cursor
        if Palmfingers == [0, 1, 1, 0, 0]:
            cv2.circle(currentSlide, IndxFinger, 10, (17, 131, 207), cv2.FILLED)

        #Fourth gesture for drawing
        if Palmfingers == [0, 0, 0, 0, 1]:
            if drawingPoint is False:
                drawingPoint = True
                drawingCount += 1
                drawing.append([])
            cv2.circle(currentSlide, IndxFinger, 10, (17, 131, 207), cv2.FILLED)
            drawing[drawingCount].append(IndxFinger)
        else:
            drawingPoint = False

# Debouncing mechanism: keyCount and keyEntered will be reset if keyCount becomes greater than keyDelay 
    if keyEntered:
        keyCount += 1
        if keyCount > keyDelay:
            keyCount = 0 
            keyEntered = False
    
    #using for loop to iterate through the points and join them together to draw
    for x in range(len(drawing)):
        for y in range(len(drawing[x])):
            if y!= 0:
                cv2.line(currentSlide, drawing[x][y - 1], drawing[x][y], (17, 131, 207), 10)

    #resizing and putting the webcam frame on top of the presentation slides
    CamSmall = cv2.resize(img, (cam_width, cam_height))
    width, height, _ = currentSlide.shape
    cam_top = 600
    cam_left = currentSlide.shape[1] - cam_width
    currentSlide[cam_top: cam_top + cam_height, cam_left: cam_left + cam_width] = CamSmall


    #Displaying the presentation slide and webcam image on the top right corner of the slides
    cv2.imshow("Machine Learning Slides", currentSlide)

    #checking if the key is pressed
    button = cv2.waitKey(1)


    #using if condition to exit the loop if 'q' is pressed
    if button == ord('e'):
        break
#the camera window will be closed/released
cam.release()
cv2.destroyAllWindows()